In [62]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [63]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=2)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
result = arxiv_tool.invoke("machine learning")
result

"Published: 2019-09-08\nTitle: Lecture Notes: Optimization for Machine Learning\nAuthors: Elad Hazan\nSummary: Lecture notes on optimization for machine learning, derived from a course at\nPrinceton University and tutorials given in MLSS, Buenos Aires, as well as\nSimons Foundation, Berkeley.\n\nPublished: 2018-11-11\nTitle: An Optimal Control View of Adversarial Machine Learning\nAuthors: Xiaojin Zhu\nSummary: I describe an optimal control view of adversarial machine learning, where the\ndynamical system is the machine learner, the input are adversarial actions, and\nthe control costs are defined by the adversary's goals to do harm and be hard\nto detect. This view encompasses many types of adversarial machine learning,\nincluding test-item attacks, training-data poisoning, and adversarial reward\nshaping. The view encourages adversarial machine learning researcher to utilize\nadvances in control theory and reinforcement learning."

In [64]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool.name

'wikipedia'

In [65]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=5,
    include_raw_content=True,
    tavily_api_key = os.getenv("tavily_key")
)

search_tool.invoke("what is the indian stock market today? Nifty and Snsex price and movement")

[{'title': 'BSE SENSEX Stock Market Index - Quote - Chart',
  'url': 'https://tradingeconomics.com/india/stock-market',
  'content': "| Indexes | Price |  |  | Day | Month | Year | Date |\n ---  ---  ---  --- |\n| NIFTY 50 | 25,795.15 |  | -96.25 | -0.37% | 3.63% | 6.68% | Oct/24 |\n| SENSEX | 84,211.88 |  | -344.52 | -0.41% | 3.76% | 6.06% | Oct/24 | [...] ### India's main stock market index, the SENSEX, fell to 84212 points on October 24, 2025, losing 0.41% from the previous session. Over the past month, the index has climbed 3.76% and is up 6.06% compared to the same time last year, according to trading on a contract for difference (CFD) that tracks this benchmark index from India. The BSE SENSEX Stock Market Index is expected to trade at 82509.52 points by the end of this quarter, according to Trading Economics global macro models and analysts [...] India's main stock market index, the SENSEX, fell to 84212 points on October 24, 2025, losing 0.41% from the previous session. Over th

In [76]:
import numexpr as ne
from langchain.tools import tool

@tool
def calculator(expr: str) -> str:
    """Perform mathematical calculations. Input should be a valid mathematical expression."""
    try:
        # numexpr provides safe evaluation (no code execution)
        result = ne.evaluate(expr)
        return f"Result: {result}"
    except Exception as e:
        return f"Calculation error: {str(e)}"

In [77]:
tools = [wiki_tool, search_tool, arxiv_tool, calculator]

In [78]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model=os.getenv("GEMINI_MODEL"),
    temperature=0,
    google_api_key=os.getenv("GOOGLE_API_KEY"),
)
llm.invoke("hey how are you?")

AIMessage(content="I'm doing great, thanks for asking! As an AI, I don't have feelings, but I'm running at 100% and ready to help.\n\nWhat can I do for you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--9784d0f2-d4f3-476c-a597-61d523da25bf-0', usage_metadata={'input_tokens': 6, 'output_tokens': 1113, 'total_tokens': 1119, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1067}})

In [79]:
llm_with_tools = llm.bind_tools(tools=tools)

In [80]:
llm_with_tools.invoke("square(sin(90)*25 + 20*cos(45))")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculator', 'arguments': '{"expr": "(sin(90*3.1415926535/180)*25 + 20*cos(45*3.1415926535/180))**2"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f9b40922-9977-486d-b86d-5881ffe6944f-0', tool_calls=[{'name': 'calculator', 'args': {'expr': '(sin(90*3.1415926535/180)*25 + 20*cos(45*3.1415926535/180))**2'}, 'id': 'a803e4ce-11a4-405a-892f-853b54f2c064', 'type': 'tool_call'}], usage_metadata={'input_tokens': 292, 'output_tokens': 379, 'total_tokens': 671, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 314}})

In [70]:
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages

In [81]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [72]:
# from Ipython.display import Image, Display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


In [82]:
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")

# tools_condition checks if the last message has tool calls
# If yes -> routes to "tools"
# If no -> routes to END
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
)

# After tools execute, go back to the LLM to process results
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

In [83]:
from langchain_core.runnables.graph import MermaidDrawMethod

# Save to file
png_data = graph.get_graph().draw_mermaid_png(
    draw_method=MermaidDrawMethod.API
)

with open("graph.png", "wb") as f:
    f.write(png_data)

In [84]:
messages = graph.invoke({"messages": [HumanMessage(content="square(sin(90)*25 + 20*cos(45))")]})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

square(sin(90)*25 + 20*cos(45))
================================== Ai Message ==================================
Tool Calls:
  calculator (b2552bb1-6f6f-4ec6-9971-7a028030fc20)
 Call ID: b2552bb1-6f6f-4ec6-9971-7a028030fc20
  Args:
    expr: (sin(90)*25 + 20*cos(45))**2
================================= Tool Message =================================
Name: calculator

Result: 1079.5401536738068
================================== Ai Message ==================================

[{'type': 'text', 'text': 'The result is 1079.54.', 'extras': {'signature': 'Cu8FAdHtim83b+D0yJuwu6+B19//cydU8TEkjxhd0Ix0r6AEHI3sr1B5gW6E03pPn8bh+OOi6tBslkg2OzKVjNQmcJp7Hk2HtvmCiLJ1++1bFa3V8iIwkgbBQn8589IN75rIb/TAdCXPYZI7BvmOY3ca5ob6MPzF26yADj6f01Xi1k8AQzVl3HpszZQ7PuhF7TfRVSSC9bpo9RfwEfgwb1HkVvDlAhFD1/WeuR8CxsLdCYLA54yMniz8daG9afYCgpsiQzvuPY/nKkP5IvPmHKQ6xRgV5k3ui3zITgBzyzdySwDZdx0wDs6zFzNodAoOW0EIuN6CL7pJGZgc0GI6/I7xWnBI6p9MtKvtZoCU9C